In [57]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime

DATA_PATH = 'data/'

In [58]:
# TODO: Make scraper module so that we can get data from inside Jupyter Notebook
# Get league data and league_standings

Once we have the data downloaded we can start to create our dataset. We will begin by using data for the last 12 seasons. This should give us enough data to make good predictions, going any further back and the data might not as relevant. 

Due to the nature of football data being time-series data (ie: matches occur over the course of a season) we will be using full seasons (or two seasons) for our test data. We can also use a repeated K-fold to check our accuracy. 

We will first load our data as seperate seasons. We are removing any rows containing NaNs and converting Date to a datetime object, we are also adding Id column.

In [59]:
def create_df(path):
    """
    Function to convert date to datetime and add 'Id' column
    """
    df = (pd.read_csv(path)
         .assign(Date=lambda df: pd.to_datetime(df.Date))
         .pipe(lambda df: df.dropna(thresh=len(df) - 2, axis=1))  # Drop cols with NAs
         .dropna(axis=0)  # Drop rows with NAs
         .sort_values('season')
         .reset_index(drop=True)
         .assign(gameId=lambda df: list(df.index + 1))
         )
    return df

In [60]:
df = create_df(os.path.join(DATA_PATH, 'all_seasons_joined.csv'))

In [61]:
# Define a function which restructures our DataFrame
def create_multiline_df_stats(old_stats_df):
    # Create a list of columns we want and their mappings to more interpretable names
    home_stats_cols = ['Date', 'season', 'HomeTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
                       'HR', 'AR']
    
    away_stats_cols = ['Date', 'season', 'AwayTeam', 'FTAG', 'FTHG', 'HTAG', 'HTHG', 'AS', 'HS', 'AST', 'HST', 'AF', 'HF', 'AC', 'HC', 'AY', 'HY',
                       'AR', 'HR']
    
    stats_cols_mapping = ['Date', 'season', 'Team', 'goalsFor', 'goalsAgainst', 'halfTimeGoalsFor', 'halfTimeGoalsAgainst', 'shotsFor',
                          'shotsAgainst', 'shotsOnTargetFor', 'shotsOnTargetAgainst', 'freesFor', 'freesAgainst', 
                          'cornersFor', 'cornersAgainst', 'yellowsFor', 'yellowsAgainst', 'redsFor', 'redsAgainst']
    
    # Create a dictionary of the old column names to new column names
    home_mapping = {old_col: new_col for old_col, new_col in zip(home_stats_cols, stats_cols_mapping)}
    away_mapping = {old_col: new_col for old_col, new_col in zip(away_stats_cols, stats_cols_mapping)}
    
    # Put each team onto an individual row
    multi_line_stats = (old_stats_df[['gameId'] + home_stats_cols] # Filter for only the home team columns
                    .rename(columns=home_mapping) # Rename the columns
                    .assign(homeGame=1) # Assign homeGame=1 so that we can use a general function later
                    .append((old_stats_df[['gameId'] + away_stats_cols]) # Append the away team columns
                            .rename(columns=away_mapping) # Rename the away team columns
                            .assign(homeGame=0), sort=True)
                    .sort_values(by='gameId') # Sort the values
                    .reset_index(drop=True))
    return multi_line_stats

In [62]:
# Define a function which creates an EMA DataFrame from the stats DataFrame
def create_stats_features_ema(stats, span):
    # Create a restructured DataFrames so that we can calculate EMA
    multi_line_stats = create_multiline_df_stats(stats)
    
    # Create a copy of the DataFrame
    ema_features = multi_line_stats[['Date', 'season', 'gameId', 'Team', 'homeGame']].copy()
    
    # Get the columns that we want to create EMA for
    feature_names = multi_line_stats.drop(columns=['Date', 'season', 'gameId', 'Team', 'homeGame']).columns
    
    # Loop over the features
    for feature_name in feature_names:
        feature_ema = (multi_line_stats.groupby('Team')[feature_name] # Calculate the EMA
                                                  .transform(lambda row: row.ewm(span=span, min_periods=2)
                                                             .mean()
                                                             .shift(1))) # Shift the data down 1 so we don't leak data
        ema_features[feature_name] = feature_ema # Add the new feature to the DataFrame
    return ema_features

In [63]:
# Then we add our weighted average to each row
df = create_stats_features_ema(df, 50)
df.tail()

,Date,season,gameId,Team,homeGame,cornersAgainst,cornersFor,freesAgainst,freesFor,goalsAgainst,...,halfTimeGoalsAgainst,halfTimeGoalsFor,redsAgainst,redsFor,shotsAgainst,shotsFor,shotsOnTargetAgainst,shotsOnTargetFor,yellowsAgainst,yellowsFor
9353,2019-08-31,1920,4677,Southampton,1,6.116358,4.912194,9.664058,10.580837,1.983081,...,0.954904,0.468097,0.115374,0.069336,13.844992,12.568356,4.845611,4.326707,1.431626,1.640568
9354,2019-09-22,1920,4678,Aston Villa,0,6.725107,4.384735,11.494342,11.178322,1.711605,...,0.658963,0.399877,0.105584,0.088425,15.152936,11.126781,4.773992,3.778736,1.691058,1.890663
9355,2019-09-22,1920,4678,Arsenal,1,5.302019,6.509403,12.031823,10.563224,1.345754,...,0.707280,0.710514,0.050738,0.023496,13.864636,13.001143,4.711074,4.694789,2.044458,1.938049
9356,2019-10-11,1920,4679,Liverpool,1,3.607329,6.698337,9.207437,8.921460,0.692422,...,0.287763,1.057772,0.016299,0.037183,8.400185,16.335754,2.579782,6.289767,1.317787,1.027593
9357,2019-10-11,1920,4679,Man City,0,2.570726,7.912103,8.222973,9.347653,0.785055,...,0.425845,1.372397,0.029012,0.061104,6.673178,19.167914,2.797690,7.057322,1.499041,1.444063


In [64]:
def restructure_stats_features(stats_features):
    non_features = ['homeGame', 'Date', 'season', 'Team', 'gameId']

    stats_features_restructured = (stats_features.query('homeGame == 1')
                                    .rename(columns={col: 'f_' + col + 'Home' for col in stats_features.columns if col not in non_features})
                                    .rename(columns={'Team': 'HomeTeam'})
                                    .pipe(pd.merge, (stats_features.query('homeGame == 0')
                                                        .rename(columns={'Team': 'AwayTeam'})
                                                        .rename(columns={col: 'f_' + col + 'Away' for col in stats_features.columns 
                                                                         if col not in non_features})), on=['gameId'])
                                    .dropna())
    return stats_features_restructured

df = restructure_stats_features(df)
df.tail()

,Date_x,season_x,gameId,HomeTeam,homeGame_x,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,...,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,f_yellowsAgainstAway,f_yellowsForAway
4674,2019-08-31,1920,4675,Crystal Palace,1,5.842739,4.736525,11.732687,11.091149,1.429201,...,0.685860,0.416198,0.109894,0.051218,14.873463,11.172772,4.764767,3.851338,1.678448,1.804567
4675,2019-08-31,1920,4676,Chelsea,1,3.582244,5.747359,11.705261,9.155159,1.122772,...,0.180516,0.445775,0.000000,0.110156,11.282876,10.490978,3.444338,3.147837,1.454426,1.918097
4676,2019-08-31,1920,4677,Southampton,1,6.116358,4.912194,9.664058,10.580837,1.983081,...,0.481550,0.902246,0.024048,0.056026,11.850505,13.667403,4.339498,5.423414,2.144057,1.973310
4677,2019-09-22,1920,4678,Arsenal,1,5.302019,6.509403,12.031823,10.563224,1.345754,...,0.658963,0.399877,0.105584,0.088425,15.152936,11.126781,4.773992,3.778736,1.691058,1.890663
4678,2019-10-11,1920,4679,Liverpool,1,3.607329,6.698337,9.207437,8.921460,0.692422,...,0.425845,1.372397,0.029012,0.061104,6.673178,19.167914,2.797690,7.057322,1.499041,1.444063


In [67]:
testdata = df.loc[df['season_x'] == 1920]

In [ ]:
# df = df[['DateHome', 'Id', 'HomeTeam', 'HomeOddsHome',
#        'DrawOddsHome', 'AwayOddsHome', 'GoalsAgainstHome',
#        'GoalsForHome', 'ShotsAgainstHome', 'ShotsForHome',
#        'AwayTeam', 'GoalsAgainstAway', 'GoalsForAway',
#        'ShotsAgainstAway', 'ShotsForAway', 'Result']]

In [68]:

df.to_csv(os.path.join(DATA_PATH, 'ema_test_data.csv'))